In [1]:
from numpy import array
import tensorflow as tf
import pandas as pd
from sklearn.metrics import mean_squared_error

In [32]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [52]:
countries = ['Austria', 'Brazil', 'China', 'Czechia', 'Denmark', 'France', 'Germany', 'India', 'Italy', 'Korea_South', 'South_Africa', 'Spain', 'UK']
results = []
for country in countries:
    result = {'country':'', 'conf': 0, 'dead': 0, 'recov': 0}
    result['country'] = country
    file = '../Data_Preprocessing/Clean/' + country + '.csv'
    df = pd.read_csv(file)
    confirmed = df['Confirmed_Cases'].tolist()
    recoveries = df['Recoveries'].tolist()
    deaths = df['Deaths'].tolist()

    n_steps = 4

    X, y = split_sequence(confirmed, n_steps)
    X_r, y_r = split_sequence(recoveries, n_steps)
    X_d, y_d = split_sequence(confirmed, n_steps)

    n_features = 1
    n_seq = 2
    n_steps = 2
    X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
    X_r = X_r.reshape((X_r.shape[0], n_seq, 1, n_steps, n_features))
    X_d = X_d.reshape((X_d.shape[0], n_seq, 1, n_steps, n_features))
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X, y, epochs=500, verbose=0)
    X_test, y_test = split_sequence(confirmed[25:48], 4)
    X_test = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
    res = model.predict(X_test)
    result['conf'] = mean_squared_error(y_test, res)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_r, y_r, epochs=500, verbose=0)
    
    X_test_r, y_test_r = split_sequence(recoveries[25:48], 4)
    X_test_r = X_test_r.reshape((X_test_r.shape[0], n_seq, 1, n_steps, n_features))
    res_r = model.predict(X_test_r)
    result['recov'] = mean_squared_error(y_test_r, res_r)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_d, y_d, epochs=500, verbose=0)
    
    X_test_d, y_test_d = split_sequence(deaths[25:48], 4)
    X_test_d = X_test_d.reshape((X_test_d.shape[0], n_seq, 1, n_steps, n_features))
    res_d = model.predict(X_test_d)
    result['dead'] = mean_squared_error(y_test_d, res_d)
    
    results.append(result)

results

[{'country': 'Austria',
  'conf': 0.7348827393049785,
  'dead': 0.009562368815707822,
  'recov': 4.641040054956178},
 {'country': 'Brazil',
  'conf': 3.2366707540935913,
  'dead': 0.05889646733845044,
  'recov': 0.24481586388464294},
 {'country': 'China',
  'conf': 1144858.7747697972,
  'dead': 1884.430386071641,
  'recov': 200654.66484423628},
 {'country': 'Czechia',
  'conf': 1.8446387192752858,
  'dead': 0.004313786672741016,
  'recov': 0.014105580996119693},
 {'country': 'Denmark',
  'conf': 2.1690301117333197,
  'dead': 0.011656146810088197,
  'recov': 1.2185416083812277},
 {'country': 'France',
  'conf': 947.35321175209,
  'dead': 6.39652906954804,
  'recov': 1526.511021328647},
 {'country': 'Germany',
  'conf': 916.767585485924,
  'dead': 0.21266168235913305,
  'recov': 11.09508983820752},
 {'country': 'India',
  'conf': 11.436985449673221,
  'dead': 0.05100822967239882,
  'recov': 0.12069719374505408},
 {'country': 'Italy',
  'conf': 23841.222747354677,
  'dead': 584.3482648573

In [46]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=500, verbose=1)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 3s 42ms/sample - loss: 167696218.2857
Epoch 2/500
77/77 [==============================] - 0s 1ms/sample - loss: 162113415.4805
Epoch 3/500
77/77 [==============================] - 0s 1ms/sample - loss: 156761271.0649
Epoch 4/500
77/77 [==============================] - 0s 1ms/sample - loss: 151337282.1818
Epoch 5/500
77/77 [==============================] - 0s 1ms/sample - loss: 143000399.5844
Epoch 6/500
77/77 [==============================] - 0s 1ms/sample - loss: 135076006.8571
Epoch 7/500
77/77 [==============================] - 0s 1ms/sample - loss: 113935399.3766
Epoch 8/500
77/77 [==============================] - 0s 1ms/sample - loss: 93238580.8831
Epoch 9/500
77/77 [==============================] - 0s 1ms/sample - loss: 71447381.8182
Epoch 10/500
77/77 [==============================] - 0s 1ms/sample - loss: 47073793.7273
Epoch 11/500
77/77 [==============================] - 0s 1ms/sample - loss: 26299

77/77 [==============================] - 0s 1ms/sample - loss: 4024019.3896
Epoch 184/500
77/77 [==============================] - 0s 1ms/sample - loss: 4343095.4286
Epoch 185/500
77/77 [==============================] - 0s 1ms/sample - loss: 5476204.6851
Epoch 186/500
77/77 [==============================] - 0s 1ms/sample - loss: 4447973.5195
Epoch 187/500
77/77 [==============================] - 0s 1ms/sample - loss: 3751316.7825
Epoch 188/500
77/77 [==============================] - 0s 1ms/sample - loss: 4214226.9578
Epoch 189/500
77/77 [==============================] - 0s 1ms/sample - loss: 3856891.3636
Epoch 190/500
77/77 [==============================] - 0s 1ms/sample - loss: 3530414.7532
Epoch 191/500
77/77 [==============================] - 0s 1ms/sample - loss: 3529309.6039
Epoch 192/500
77/77 [==============================] - 0s 1ms/sample - loss: 3746632.0000
Epoch 193/500
77/77 [==============================] - 0s 1ms/sample - loss: 3569851.6104
Epoch 194/500
77/77 [===

77/77 [==============================] - 0s 1ms/sample - loss: 2821974.5584
Epoch 366/500
77/77 [==============================] - 0s 1ms/sample - loss: 2469213.4221
Epoch 367/500
77/77 [==============================] - 0s 1ms/sample - loss: 1930272.8724
Epoch 368/500
77/77 [==============================] - 0s 1ms/sample - loss: 3443132.4675
Epoch 369/500
77/77 [==============================] - 0s 1ms/sample - loss: 3777055.1558
Epoch 370/500
77/77 [==============================] - 0s 1ms/sample - loss: 3117499.7532
Epoch 371/500
77/77 [==============================] - 0s 1ms/sample - loss: 3302273.7727
Epoch 372/500
77/77 [==============================] - 0s 1ms/sample - loss: 3082364.3194
Epoch 373/500
77/77 [==============================] - 0s 1ms/sample - loss: 2843740.2800
Epoch 374/500
77/77 [==============================] - 0s 1ms/sample - loss: 2978583.7971
Epoch 375/500
77/77 [==============================] - 0s 1ms/sample - loss: 2767855.7662
Epoch 376/500
77/77 [===

In [47]:
X_test, y_test = split_sequence(confirmed[25:48], 4)
X_test = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
res = model.predict(X_test)
mean_squared_error(y_test, res)

628.3510212387673

In [40]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_r, y_r, epochs=500, verbose=1)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 3s 42ms/sample - loss: 1073685.4706
Epoch 2/500
77/77 [==============================] - 0s 1ms/sample - loss: 1030930.5682
Epoch 3/500
77/77 [==============================] - 0s 1ms/sample - loss: 1008850.6071
Epoch 4/500
77/77 [==============================] - 0s 1ms/sample - loss: 977911.0211
Epoch 5/500
77/77 [==============================] - 0s 1ms/sample - loss: 952655.9481
Epoch 6/500
77/77 [==============================] - 0s 1ms/sample - loss: 909876.2898
Epoch 7/500
77/77 [==============================] - 0s 1ms/sample - loss: 874160.5406
Epoch 8/500
77/77 [==============================] - 0s 1ms/sample - loss: 821148.5162
Epoch 9/500
77/77 [==============================] - 0s 1ms/sample - loss: 763694.5649
Epoch 10/500
77/77 [==============================] - 0s 1ms/sample - loss: 700249.6510
Epoch 11/500
77/77 [==============================] - 0s 1ms/sample - loss: 636467.3293
Epoch 12/500
77/7

77/77 [==============================] - 0s 1ms/sample - loss: 104649.4330
Epoch 188/500
77/77 [==============================] - 0s 1ms/sample - loss: 102497.9218
Epoch 189/500
77/77 [==============================] - 0s 1ms/sample - loss: 105577.7353
Epoch 190/500
77/77 [==============================] - 0s 1ms/sample - loss: 133770.2993
Epoch 191/500
77/77 [==============================] - 0s 1ms/sample - loss: 141515.8929
Epoch 192/500
77/77 [==============================] - 0s 1ms/sample - loss: 149952.7656
Epoch 193/500
77/77 [==============================] - 0s 1ms/sample - loss: 154865.1203
Epoch 194/500
77/77 [==============================] - 0s 1ms/sample - loss: 164626.6672
Epoch 195/500
77/77 [==============================] - 0s 1ms/sample - loss: 133579.8736
Epoch 196/500
77/77 [==============================] - 0s 1ms/sample - loss: 127088.7406
Epoch 197/500
77/77 [==============================] - 0s 1ms/sample - loss: 113285.2530
Epoch 198/500
77/77 [==============

77/77 [==============================] - 0s 1ms/sample - loss: 45058.8856
Epoch 373/500
77/77 [==============================] - 0s 1ms/sample - loss: 44777.5697
Epoch 374/500
77/77 [==============================] - 0s 1ms/sample - loss: 44576.3758
Epoch 375/500
77/77 [==============================] - 0s 1ms/sample - loss: 44311.2413
Epoch 376/500
77/77 [==============================] - 0s 1ms/sample - loss: 44064.5911
Epoch 377/500
77/77 [==============================] - 0s 1ms/sample - loss: 43827.1950
Epoch 378/500
77/77 [==============================] - 0s 1ms/sample - loss: 43555.9225
Epoch 379/500
77/77 [==============================] - 0s 1ms/sample - loss: 43410.9651
Epoch 380/500
77/77 [==============================] - 0s 1ms/sample - loss: 43011.3451
Epoch 381/500
77/77 [==============================] - 0s 1ms/sample - loss: 42917.8749
Epoch 382/500
77/77 [==============================] - 0s 1ms/sample - loss: 42577.7874
Epoch 383/500
77/77 [=========================

In [41]:
X_test_r, y_test_r = split_sequence(recoveries[25:48], 4)
X_test_r = X_test_r.reshape((X_test_r.shape[0], n_seq, 1, n_steps, n_features))
res_r = model.predict(X_test_r)
mean_squared_error(y_test_r, res_r)

7.325915214097329

In [42]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=128, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_d, y_d, epochs=500, verbose=1)

Train on 77 samples
Epoch 1/500
77/77 [==============================] - 3s 44ms/sample - loss: 149520244.3636
Epoch 2/500
77/77 [==============================] - 0s 1ms/sample - loss: 144026453.5065
Epoch 3/500
77/77 [==============================] - 0s 1ms/sample - loss: 135916004.7792
Epoch 4/500
77/77 [==============================] - 0s 1ms/sample - loss: 125499537.6623
Epoch 5/500
77/77 [==============================] - 0s 1ms/sample - loss: 108600145.5584
Epoch 6/500
77/77 [==============================] - 0s 1ms/sample - loss: 91583809.0390
Epoch 7/500
77/77 [==============================] - 0s 1ms/sample - loss: 70455689.6623
Epoch 8/500
77/77 [==============================] - 0s 1ms/sample - loss: 51684581.8701
Epoch 9/500
77/77 [==============================] - 0s 1ms/sample - loss: 27324012.6753
Epoch 10/500
77/77 [==============================] - 0s 1ms/sample - loss: 13075884.6623
Epoch 11/500
77/77 [==============================] - 0s 1ms/sample - loss: 5944628

77/77 [==============================] - 0s 1ms/sample - loss: 3879944.8815
Epoch 184/500
77/77 [==============================] - 0s 1ms/sample - loss: 3817781.2078
Epoch 185/500
77/77 [==============================] - 0s 1ms/sample - loss: 3751713.4221
Epoch 186/500
77/77 [==============================] - 0s 1ms/sample - loss: 4065129.7403
Epoch 187/500
77/77 [==============================] - 0s 1ms/sample - loss: 3778008.5519
Epoch 188/500
77/77 [==============================] - 0s 1ms/sample - loss: 3779479.2597
Epoch 189/500
77/77 [==============================] - 0s 1ms/sample - loss: 3754000.5325
Epoch 190/500
77/77 [==============================] - 0s 1ms/sample - loss: 3759154.0651
Epoch 191/500
77/77 [==============================] - 0s 1ms/sample - loss: 3741474.0519
Epoch 192/500
77/77 [==============================] - 0s 1ms/sample - loss: 3712141.6039
Epoch 193/500
77/77 [==============================] - 0s 1ms/sample - loss: 3770495.6753
Epoch 194/500
77/77 [===

77/77 [==============================] - 0s 1ms/sample - loss: 2557567.3149
Epoch 366/500
77/77 [==============================] - 0s 1ms/sample - loss: 2673310.1234
Epoch 367/500
77/77 [==============================] - 0s 1ms/sample - loss: 2712552.2175
Epoch 368/500
77/77 [==============================] - 0s 1ms/sample - loss: 2549755.1201
Epoch 369/500
77/77 [==============================] - 0s 1ms/sample - loss: 2598377.6818
Epoch 370/500
77/77 [==============================] - 0s 1ms/sample - loss: 2728964.4838
Epoch 371/500
77/77 [==============================] - 0s 1ms/sample - loss: 2619870.2403
Epoch 372/500
77/77 [==============================] - 0s 1ms/sample - loss: 2579506.3734
Epoch 373/500
77/77 [==============================] - 0s 1ms/sample - loss: 2631607.8636
Epoch 374/500
77/77 [==============================] - 0s 1ms/sample - loss: 2694962.0779
Epoch 375/500
77/77 [==============================] - 0s 1ms/sample - loss: 2548079.1464
Epoch 376/500
77/77 [===

In [44]:
X_test_d, y_test_d = split_sequence(deaths[25:48], 4)
X_test_d = X_test_d.reshape((X_test_d.shape[0], n_seq, 1, n_steps, n_features))
res_d = model.predict(X_test_d)
mean_squared_error(y_test_d, res_d)

16.106315840520487